In [ ]:
import os

import gym
import numpy as np
import matplotlib.pyplot as plt

from stable_baselines.bench import Monitor
from stable_baselines.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines.common.policies import MlpPolicy, FeedForwardPolicy
from stable_baselines import results_plotter
from stable_baselines import *

In [ ]:
import gym
import numpy as np

# Create log dir
log_dir = "tmp/"
os.makedirs(log_dir, exist_ok=True)

# Customize value network 
class CustomPolicy(FeedForwardPolicy):
    def __init__(self, *args, **kwargs):
        super(CustomPolicy, self).__init__(*args, **kwargs,
                                           net_arch=[dict(vf=[10, 7, 5])],
                                           feature_extraction="mlp")
        
# Create and wrap the environment
env = gym.make('Taxi-v3')
env = Monitor(env, log_dir, allow_early_resets=True)
env = DummyVecEnv(([lambda: env]))

model = TRPO(CustomPolicy, env, verbose=1)

# Train the agent
time_steps = 100000
model.learn(total_timesteps=time_steps)

In [ ]:
results_plotter.plot_results([log_dir], time_steps, results_plotter.X_EPISODES, "TRPO")
plt.show()

In [ ]:
obs = env.reset()
for i in range(199):
    action, _states = model.predict(obs)
    obs, rewards, dones, info = env.step(action)
    env.render()